# Notebook 5: Vector Database (Chroma) e Chunking Avanzato

**Obiettivo**: Implementare un sistema RAG completo usando Chroma (vector database), chunking avanzato e retrieval ottimizzato

---


## 1. Setup e Import

Importiamo tutte le librerie necessarie per Chroma, document loading, chunking e RAG.


In [1]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Chroma Vector Database
from langchain_chroma import Chroma

# Document Loaders
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader

# Text Splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Utilità
import os
import shutil

model="llama3.2:3b"
embedder = "all-minilm:l6-v2" # nomic-embed-text

# Inizializza LLM (chat model per generazione)
llm = ChatOllama(
    model=model,
    temperature=0.7
)

# Inizializza Embeddings
embeddings = OllamaEmbeddings(
    model=embedder
)

print("✅ Setup completato!")
print(f"LLM: {model}")
print(f"Embeddings: {embedder}")
print(f"Vector DB: Chroma")


✅ Setup completato!
LLM: llama3.2:3b
Embeddings: all-minilm:l6-v2
Vector DB: Chroma


## 2. Preparazione Documenti di Esempio

Creiamo documenti di esempio per testare il sistema RAG. In produzione, questi verrebbero caricati da file reali.


In [2]:
# Crea directory per documenti di esempio
os.makedirs("documenti_esempio", exist_ok=True)

# Documento 1: FAQ Ritiro Documenti
doc1_content = """FAQ - Ritiro Documenti d'Identità

Il documento d'identità può essere ritirato presso l'ufficio anagrafe del comune.

ORARI DI RITIRO:
- Dal lunedì al venerdì: 9:00 - 13:00
- Martedì e giovedì anche pomeriggio: 15:00 - 17:30
- Chiuso sabato, domenica e giorni festivi

DOCUMENTI NECESSARI:
- Ricevuta del pagamento (bollettino o PagoPA)
- Documento di riconoscimento valido (carta d'identità scaduta, passaporto, patente)
- Codice fiscale

TEMPI:
Il documento viene rilasciato entro 15 giorni lavorativi dalla richiesta.
In caso di urgenza, è possibile richiedere il documento temporaneo valido 3 mesi.

COSTI:
- Carta d'identità elettronica: €22,20
- Documento temporaneo: €5,16
"""

# Documento 2: FAQ Orari Uffici
doc2_content = """FAQ - Orari e Contatti Uffici Comunali

ORARI GENERALI:
Gli uffici comunali sono aperti al pubblico con i seguenti orari:
- Lunedì, Mercoledì, Venerdì: 9:00 - 13:00
- Martedì e Giovedì: 9:00 - 13:00 e 15:00 - 17:30
- Sabato, Domenica e Festivi: CHIUSO

UFFICI PRINCIPALI:
- Ufficio Anagrafe: piano terra, stanza 101
- Ufficio Tributi: piano terra, stanza 102
- Ufficio Protocollo: piano terra, stanza 103
- Ufficio SUAP: primo piano, stanza 201

CONTATTI:
- Telefono: 091-1234567
- Email: info@comune.esempio.it
- PEC: comune@pec.esempio.it

PRENOTAZIONI:
È possibile prenotare un appuntamento online tramite il portale del comune o telefonando al numero verde 800-123456.
"""

# Documento 3: FAQ Certificati Online
doc3_content = """FAQ - Certificati Online

È possibile richiedere certificati online tramite il portale del comune utilizzando SPID o CIE.

TIPI DI CERTIFICATI DISPONIBILI:
- Certificato di residenza
- Certificato di stato di famiglia
- Certificato di nascita
- Certificato di matrimonio
- Certificato di cittadinanza
- Certificato anagrafico generale

TEMPI DI EMISSIONE:
I certificati vengono emessi entro 3 giorni lavorativi dalla richiesta e inviati via:
- Email (se richiesto)
- PEC (Posta Elettronica Certificata)
- Ritiro presso sportello (se richiesto)

COSTI:
- Certificati online: GRATUITI
- Certificati cartacei presso sportello: €0,52 per marca da bollo

AUTENTICAZIONE:
Per accedere al servizio è necessario:
- SPID (Sistema Pubblico di Identità Digitale) livello 2 o 3
- CIE (Carta d'Identità Elettronica) con PIN
"""

# Salva documenti come file di testo
with open("documenti_esempio/faq_ritiro_documenti.txt", "w", encoding="utf-8") as f:
    f.write(doc1_content)

with open("documenti_esempio/faq_orari_uffici.txt", "w", encoding="utf-8") as f:
    f.write(doc2_content)

with open("documenti_esempio/faq_certificati_online.txt", "w", encoding="utf-8") as f:
    f.write(doc3_content)

print("✅ Documenti di esempio creati:")
print("  - documenti_esempio/faq_ritiro_documenti.txt")
print("  - documenti_esempio/faq_orari_uffici.txt")
print("  - documenti_esempio/faq_certificati_online.txt")


✅ Documenti di esempio creati:
  - documenti_esempio/faq_ritiro_documenti.txt
  - documenti_esempio/faq_orari_uffici.txt
  - documenti_esempio/faq_certificati_online.txt


## 3. Caricamento Documenti

Carichiamo i documenti usando i document loaders di LangChain.

TextLoader è uno dei componenti fondamentali e più semplici di LangChain.
Serve a prendere un file di testo grezzo (.txt) e a trasformarlo in un oggetto che LangChain può elaborare.


In [3]:
# Carica documenti usando TextLoader
documents = []

# Carica ogni documento
for filename in ["faq_ritiro_documenti.txt", "faq_orari_uffici.txt", "faq_certificati_online.txt"]:
    filepath = f"documenti_esempio/{filename}"
    loader = TextLoader(filepath, encoding="utf-8")
    docs = loader.load()
    
    # Aggiungi metadata per identificare la fonte
    for doc in docs:
        doc.metadata["source"] = filename
        doc.metadata["type"] = "FAQ"
    
    documents.extend(docs)

print(f"✅ Caricati {len(documents)} documenti")
print(f"\nDocumenti caricati:")
for i, doc in enumerate(documents, 1):
    print(f"  {i}. {doc.metadata['source']} ({len(doc.page_content)} caratteri)")


✅ Caricati 3 documenti

Documenti caricati:
  1. faq_ritiro_documenti.txt (680 caratteri)
  2. faq_orari_uffici.txt (673 caratteri)
  3. faq_certificati_online.txt (811 caratteri)


## 4. Chunking Avanzato

Dividiamo i documenti in chunk più piccoli usando RecursiveCharacterTextSplitter. Questo è cruciale per:
- Migliorare la precisione del retrieval
- Gestire documenti lunghi
- Ottimizzare l'uso del context window dell'LLM


In [4]:
# Configurazione chunking
# RecursiveCharacterTextSplitter divide ricorsivamente per:
# 1. Paragrafi (doppio newline)
# 2. Frasi (punto)
# 3. Parole (spazio)
# Questo rispetta la struttura del testo

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Dimensione massima chunk (caratteri)
    chunk_overlap=50,      # Overlap tra chunk (mantiene contesto)
    length_function=len,   # Funzione per calcolare lunghezza
    separators=["\n\n", "\n", ". ", " ", ""]  # Delimitatori in ordine di priorità
)

# Applica chunking ai documenti
chunks = text_splitter.split_documents(documents)

print(f"✅ Chunking completato")
print(f"Documenti originali: {len(documents)}")
print(f"Chunk creati: {len(chunks)}")
print(f"\nEsempio chunk:")
print(f"  Chunk 1: {chunks[0].page_content[:100]}...")
print(f"  Metadata: {chunks[0].metadata}")
print(f"  Lunghezza: {len(chunks[0].page_content)} caratteri")


✅ Chunking completato
Documenti originali: 3
Chunk creati: 6

Esempio chunk:
  Chunk 1: FAQ - Ritiro Documenti d'Identità

Il documento d'identità può essere ritirato presso l'ufficio anag...
  Metadata: {'source': 'faq_ritiro_documenti.txt', 'type': 'FAQ'}
  Lunghezza: 438 caratteri


## 5. Creazione Chroma Vector Store

Creiamo il vector database Chroma e inseriamo i chunk con i loro embedding.


In [5]:
# Rimuovi vector store esistente se presente (per test)
if os.path.exists("./chroma_db"):
    shutil.rmtree("./chroma_db")
    print("🗑️ Vector store esistente rimosso")

# Crea Chroma vector store dai chunk
# Chroma salva automaticamente i dati su disco
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db",  # Directory per persistenza
    collection_name="faq_comunali"     # Nome collezione
)

print("✅ Chroma vector store creato!")
print(f"  Directory: ./chroma_db")
print(f"  Collezione: faq_comunali")
print(f"  Chunk inseriti: {len(chunks)}")
print(f"\nVector store è persistente: i dati sono salvati su disco")


✅ Chroma vector store creato!
  Directory: ./chroma_db
  Collezione: faq_comunali
  Chunk inseriti: 6

Vector store è persistente: i dati sono salvati su disco


## 6. Retrieval con Similarity Search

Ora che abbiamo il vector store, possiamo fare retrieval per trovare i chunk più rilevanti per una query.


In [ ]:
# Crea retriever dal vector store
# Il retriever gestisce automaticamente similarity search
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Tipo di ricerca: similarity (default)
    search_kwargs={"k": 3}      # Numero di chunk da recuperare (top-k)
)

# Test retrieval
query = "Quando posso ritirare il documento d'identità?"
print(f"Query: {query}\n")

# Recupera chunk rilevanti
relevant_chunks = retriever.invoke(query)

print(f"✅ Recuperati {len(relevant_chunks)} chunk rilevanti\n")
for i, chunk in enumerate(relevant_chunks, 1):
    print(f"Chunk {i}:")
    print(f"  Fonte: {chunk.metadata.get('source', 'N/A')}")
    print(f"  Contenuto: {chunk.page_content[:150]}...")
    print()


Query: Quando posso ritirare il documento d'identità?

✅ Recuperati 3 chunk rilevanti

Chunk 1:
  Fonte: faq_ritiro_documenti.txt
  Contenuto: FAQ - Ritiro Documenti d'Identità

Il documento d'identità può essere ritirato presso l'ufficio anagrafe del comune.

ORARI DI RITIRO:
- Dal lunedì al...

Chunk 2:
  Fonte: faq_ritiro_documenti.txt
  Contenuto: TEMPI:
Il documento viene rilasciato entro 15 giorni lavorativi dalla richiesta.
In caso di urgenza, è possibile richiedere il documento temporaneo va...

Chunk 3:
  Fonte: faq_orari_uffici.txt
  Contenuto: FAQ - Orari e Contatti Uffici Comunali

ORARI GENERALI:
Gli uffici comunali sono aperti al pubblico con i seguenti orari:
- Lunedì, Mercoledì, Venerdì...



## 7. RAG Completo

Combiniamo retrieval e generation usando LangChain per creare una pipeline RAG completa e ottimizzata.


In [ ]:
# Definisci prompt template per RAG
# Il prompt include:
# - System message con istruzioni
# - Context placeholder (verrà riempito con chunk rilevanti)
# - Query placeholder (domanda utente)

prompt = ChatPromptTemplate.from_messages([
    ("system", """Sei un assistente che risponde alle domande basandoti SOLO sul contesto fornito.
Rispondi in modo chiaro e conciso.
Se la risposta non è nel contesto, dì 'Non ho informazioni sufficienti nel contesto fornito.'
Cita sempre la fonte quando possibile."""),
    ("human", """Contesto:
{context}

Domanda: {question}

Risposta:""")
])

# Crea RAG chain usando LCEL
# La chain combina:
# 1. Retrieval: trova chunk rilevanti
# 2. Format: formatta contesto
# 3. Prompt: costruisce prompt con contesto
# 4. LLM: genera risposta
# 5. Output parser: estrae testo

def format_docs(docs):
    """Formatta i documenti recuperati in un unico contesto"""
    return "\n\n".join([
        f"Fonte: {doc.metadata.get('source', 'N/A')}\nContenuto: {doc.page_content}"
        for doc in docs
    ])

# Costruisci chain con LCEL
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG chain creato con LCEL!")
print("La chain combina: Retrieval → Format → Prompt → LLM → Output")


✅ RAG chain creato con LCEL!
La chain combina: Retrieval → Format → Prompt → LLM → Output


## 8. Test RAG Completo

Testiamo il sistema RAG completo con diverse query per verificare che funzioni correttamente.


In [ ]:
# Test con diverse query
test_queries = [
    "Quando posso ritirare il documento d'identità?",
    "Quali sono gli orari degli uffici comunali?",
    "Come posso richiedere un certificato online?",
    "Quanto costa la carta d'identità?",
    "Quali documenti servono per il ritiro?"
]

print("=== Test RAG Completo ===\n")

for i, query in enumerate(test_queries, 1):
    print(f"{'='*60}")
    print(f"Query {i}: {query}\n")
    
    # Esegui RAG chain
    response = rag_chain.invoke(query)
    
    print(f"Risposta: {response}\n")
    
    # Mostra anche i chunk recuperati (per debug)
    chunks = retriever.invoke(query)
    print(f"Chunk utilizzati: {len(chunks)}")
    print(f"Fonti: {', '.join([chunk.metadata.get('source', 'N/A') for chunk in chunks])}\n")
    print()


=== Test RAG Completo ===

Query 1: Quando posso ritirare il documento d'identità?

Risposta: Secondo la FAQ del ritiro dei documenti d'identità, puoi ritirare il documento presso l'ufficio anagrafe del comune durante gli orari di apertura specificati:

*   Dal lunedì al venerdì: 9:00 - 13:00
*   Martedì e giovedì anche pomeriggio: 15:00 - 17:30

Assicurati di portare con te la ricevuta del pagamento, il documento di riconoscimento valido e il codice fiscale.

Chunk utilizzati: 3
Fonti: faq_ritiro_documenti.txt, faq_ritiro_documenti.txt, faq_orari_uffici.txt


Query 2: Quali sono gli orari degli uffici comunali?

Risposta: Gli uffici comunali sono aperti al pubblico con i seguenti orari:

* Lunedì, Mercoledì, Venerdì: 9:00 - 13:00
* Martedì e Giovedì: 9:00 - 13:00 e 15:00 - 17:30
* Sabato, Domenica e Festivi: CHIUSO

Fonte: faq_orari_uffici.txt

Chunk utilizzati: 3
Fonti: faq_orari_uffici.txt, faq_ritiro_documenti.txt, faq_ritiro_documenti.txt


Query 3: Come posso richiedere un certif

## 9. Retrieval Ottimizzato con Filtri

Possiamo migliorare il retrieval usando filtri sui metadata per restringere la ricerca a documenti specifici.


In [ ]:
# Crea retriever con filtri metadata
# Esempio: cerca solo nei documenti di tipo "FAQ"
retriever_filtered = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,
        "filter": {"type": "FAQ"}  # Filtro per metadata
    }
)

# Test con filtro
query = "Orari uffici"
print(f"Query: {query}\n")
print("Retrieval SENZA filtro:")
chunks_no_filter = retriever.invoke(query)
for chunk in chunks_no_filter:
    print(f"  - {chunk.metadata.get('source')}")

print("\nRetrieval CON filtro (solo FAQ):")
chunks_filtered = retriever_filtered.invoke(query)
for chunk in chunks_filtered:
    print(f"  - {chunk.metadata.get('source')}")

print("\n✅ I filtri permettono di restringere la ricerca a documenti specifici")


Query: Orari uffici

Retrieval SENZA filtro:
  - faq_orari_uffici.txt
  - faq_ritiro_documenti.txt
  - faq_ritiro_documenti.txt

Retrieval CON filtro (solo FAQ):
  - faq_orari_uffici.txt
  - faq_ritiro_documenti.txt
  - faq_ritiro_documenti.txt

✅ I filtri permettono di restringere la ricerca a documenti specifici


## 10. Caricamento Vector Store Esistente

In produzione, il vector store viene creato una volta e poi caricato quando necessario. Vediamo come caricare un vector store esistente.


In [ ]:
# Carica vector store esistente (senza ricreare embedding)
# Utile quando il vector store è già stato creato e vuoi solo usarlo

vectorstore_loaded = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings,
    collection_name="faq_comunali"
)

# Verifica che funzioni
retriever_loaded = vectorstore_loaded.as_retriever(search_kwargs={"k": 2})
test_chunks = retriever_loaded.invoke("orari uffici")

print("✅ Vector store caricato con successo!")
print(f"Chunk recuperati: {len(test_chunks)}")
print(f"Fonte: {test_chunks[0].metadata.get('source')}")

print("\n💡 In produzione:")
print("  - Crea vector store una volta (indexing)")
print("  - Carica vector store quando serve (query)")
print("  - Aggiorna solo quando documenti cambiano")


✅ Vector store caricato con successo!
Chunk recuperati: 2
Fonte: faq_orari_uffici.txt

💡 In produzione:
  - Crea vector store una volta (indexing)
  - Carica vector store quando serve (query)
  - Aggiorna solo quando documenti cambiano


## 11. Confronto Strategie Chunking

Testiamo diverse strategie di chunking per vedere come influenzano il retrieval.


In [ ]:
# Strategia 1: Chunk piccoli (200 caratteri)
splitter_small = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Strategia 2: Chunk medi (500 caratteri) - quella che usiamo
splitter_medium = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Strategia 3: Chunk grandi (1000 caratteri)
splitter_large = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Test con un documento
test_doc = documents[0]  # Primo documento

chunks_small = splitter_small.split_documents([test_doc])
chunks_medium = splitter_medium.split_documents([test_doc])
chunks_large = splitter_large.split_documents([test_doc])

print("=== Confronto Strategie Chunking ===\n")
print(f"Documento originale: {len(test_doc.page_content)} caratteri\n")
print(f"Chunk piccoli (200): {len(chunks_small)} chunk")
print(f"  - Media caratteri: {sum(len(c.page_content) for c in chunks_small) / len(chunks_small):.0f}")
print(f"\nChunk medi (500): {len(chunks_medium)} chunk")
print(f"  - Media caratteri: {sum(len(c.page_content) for c in chunks_medium) / len(chunks_medium):.0f}")
print(f"\nChunk grandi (1000): {len(chunks_large)} chunk")
print(f"  - Media caratteri: {sum(len(c.page_content) for c in chunks_large) / len(chunks_large):.0f}")

print("\n💡 Considerazioni:")
print("  - Chunk piccoli: più precisione, più chunk da gestire")
print("  - Chunk medi: buon compromesso (raccomandato)")
print("  - Chunk grandi: meno chunk, ma meno precisione")


=== Confronto Strategie Chunking ===

Documento originale: 680 caratteri

Chunk piccoli (200): 5 chunk
  - Media caratteri: 134

Chunk medi (500): 2 chunk
  - Media caratteri: 338

Chunk grandi (1000): 1 chunk
  - Media caratteri: 679

💡 Considerazioni:
  - Chunk piccoli: più precisione, più chunk da gestire
  - Chunk medi: buon compromesso (raccomandato)
  - Chunk grandi: meno chunk, ma meno precisione


## Note e Best Practices

### Cosa abbiamo imparato:
1. **Chroma**: Vector database persistente, semplice da usare, perfetto per prototipi
2. **Chunking**: RecursiveCharacterTextSplitter rispetta struttura testo
3. **Retrieval**: Similarity search trova documenti rilevanti semanticamente
4. **RAG con LCEL**: Pipeline elegante e componibile
5. **Metadata**: Filtri permettono ricerca mirata
6. **Persistenza**: Vector store salvato su disco, riutilizzabile

### Best Practices:

**Chunking**:
- Dimensione chunk: 200-500 caratteri è ideale
- Overlap: 10-20% (50-100 caratteri per chunk 500)
- Delimitatori: rispettare struttura documento (paragrafi → frasi)

**Retrieval**:
- Top-k: 3-5 chunk è solitamente sufficiente
- Filtri metadata: usare per restringere ricerca
- Test con query reali per ottimizzare

**Vector Store**:
- Creare una volta (indexing phase)
- Caricare quando serve (query phase)
- Aggiornare solo quando documenti cambiano

**RAG Chain**:
- Prompt chiaro: istruzioni su uso contesto
- Gestione della "non conoscenza": comunicare espressamente quando una informazione non è disponibile
- Citazioni: includere fonti quando possibile

### Limitazioni e Considerazioni:
- **Chroma locale**: Non scala a milioni di documenti
- **Chunking**: La strategia dipende da tipo documento
- **Retrieval**: Similarity search non sempre perfetto

---

**Congratulazioni! Hai completato il Notebook 5! 🎉**
